<a href="https://colab.research.google.com/github/armandoordonez/AI-Engineering/blob/main/agentes_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain langchain_google_genai langchain_community langgraph langsmith langgraph-checkpoint-sqlite python-dotenv wikipedia youtube_search

In [ ]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "colab-agent"

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits.load_tools import get_all_tool_names
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
import os
from langsmith import traceable


In [ ]:
# Lee la clave desde la variable de entorno (usa getpass() antes)
g_api_key = os.getenv("GOOGLE_API_KEY")

# Inicialización
chat_model = ChatGoogleGenerativeAI(api_key=g_api_key, model="gemini-2.5-flash")

# Prueba rápida (mensaje simple)
resp = chat_model.invoke([HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")])
print(resp.content)

La novela narra la saga de siete generaciones de la familia Buendía, fundadores del mítico pueblo de Macondo, desde su creación hasta su inevitable desaparición. A través de un torbellino de eventos fantásticos y realistas, la estirpe está marcada por la soledad, el incesto y la repetición cíclica de su destino, culminando en su olvido total.


In [ ]:
messages = [SystemMessage(content='Eres un milenial colombiano'),
           HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")]

resp = chat_model.invoke(messages)
print(resp.content)

¡Uff, Gabo! Pues mira, la vuelta es que es la saga de los Buendía, una familia que funda el mítico Macondo y vive toda clase de vainas raras y mágicas por generaciones. Y, al final, entre la soledad, los errores que se repiten y un destino medio trágico, tanto la familia como Macondo terminan condenados a desaparecer.


In [ ]:
pelicula = "Matrix"
template = "Cuentame sobre la pelicula {pelicula} en dos frases."
prompt = PromptTemplate(input_variables=["pelicula"], template=template)

chain = prompt | chat_model
out = chain.invoke({"pelicula": pelicula})
print(out)

content='Un programador llamado Neo descubre que la realidad que conoce es una sofisticada simulación creada por máquinas inteligentes para mantener a la humanidad esclavizada. Es reclutado por un grupo de rebeldes para luchar contra ellas y liberar la mente humana, enfrentándose a la verdad sobre su existencia y su potencial.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--9afaa378-b1c1-41d8-8c86-8cb6656d8d6f-0' usage_metadata={'input_tokens': 13, 'output_tokens': 224, 'total_tokens': 237, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 163}}


In [ ]:
get_all_tool_names
get_all_tool_names()[:10]

['sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-books',
 'google-lens']

In [ ]:
wiki_wrap = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)

wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrap, description="Resumen enciclopédico")

yt_tool = YouTubeSearchTool(description="Busca videos relevantes")

tools = [wiki_tool, yt_tool]

In [ ]:
model_with_tools = chat_model.bind_tools(tools)

# Pregunta general

print("Respuesta 1")
res1 = model_with_tools.invoke([HumanMessage(content="Hola")])
print("Texto:", res1.content)
print("Tool calls sugeridas:", res1.tool_calls)


Respuesta 1
Texto: [{'type': 'text', 'text': 'Hola, ¿en qué puedo ayudarte?', 'extras': {'signature': 'Co4BAdHtim/I9tZcKJSN14vohkJRDQtZYPyg5Mh3SADdmbgXwNeaTdyB/L/U8Pe5rCaLZt8OgCM8+7FHzW2ARi4JcyibUtI12qGTpFp+PTx8HpHYj6O7u5BpJKh8TygKP823r1EmU2z1FJXgpXOQR/zAt0U/mE+6mdtyt96n26OTCML2lhwYnM1RcyoCSPEOBA=='}}]
Tool calls sugeridas: []


In [ ]:

#Pregunta específica
print("\nRespuesta 2")
res2 = model_with_tools.invoke([
   HumanMessage("Busca la biografía de Simon Bolivar")
])
print("Texto:", res2.content)
print("Tool calls sugeridas:", res2.tool_calls)


Respuesta 2
Texto: 
Tool calls sugeridas: [{'name': 'wikipedia', 'args': {'query': 'Simon Bolivar'}, 'id': '4b2b331b-2d4a-4851-a247-24a7d00a9e79', 'type': 'tool_call'}]


In [ ]:
system_prompt = SystemMessage("Eres un asistente que elige entre texto, wiki o video según convenga.")

agent = create_react_agent(chat_model, tools, prompt=system_prompt)

# Función ejecutora básica
def execute(agent, query):
    response = agent.invoke({"messages": [HumanMessage(query)]})
    for m in response["messages"]:
        print(f"{m.__class__.__name__}: {m.content}\n{'-'*20}")
    return response

# Ejemplo
resp = execute(agent, "Explícame la serie de Fourier")

/tmp/ipython-input-2676680128.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(chat_model, tools, prompt=system_prompt)


HumanMessage: Explícame la serie de Fourier
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Fourier series
Summary: A Fourier series () is a series expansion of a periodic function into a sum of trigonometric functions. The Fourier series is an example of a trigonometric series. By expressing a function as a sum of sines and cosines, many problems involving the function become easier
--------------------
AIMessage: La serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). Esto significa que puedes descomponer una señal compleja en sus componentes de frecuencia más simples. Es muy útil para analizar y resolver problemas que involucran funciones periódicas.
--------------------


In [ ]:
execute(agent, "qué recuerdas de mi anterior pregunta?")

HumanMessage: qué recuerdas de mi anterior pregunta?
--------------------
AIMessage: [{'type': 'text', 'text': 'Lo siento, no tengo memoria de conversaciones anteriores. Cada pregunta es un nuevo comienzo.', 'extras': {'signature': 'CoQCAdHtim96BEr7xTeUIwcE5DxvT9OKGZnrnevOiMTXHQhd5SDE6ibXNZH7gWuOBWIoJaGUthfXMQ/9omAReXMgysW1O78NNyXravlTvB4F092OIKTcSKAdnZ8SkEqeydPJTUyBFB9+QPAkDSjF42LnZoq7Bnv8zj8CD/MtOh4c4jUtu5UF7EhQhMc0yMvv7IEv1hBWXT1+xbatn0yamo1pQGEtTN+Mj+fnoAXhoy2dJ7rNlZj9wnVkZt0VjTDUo/OtHqimJUhEGw4b0ziJ9ZtvsVceIydJye2LDO4sAlFIrfdas5BLlPXgjqZ5pqYYjPFacRVG5TY1Pq5K/niTswkb3rDm0Oc='}}]
--------------------


{'messages': [HumanMessage(content='qué recuerdas de mi anterior pregunta?', additional_kwargs={}, response_metadata={}, id='6110046e-0c0d-48f1-86e0-73bb3ac09b30'),
  AIMessage(content=[{'type': 'text', 'text': 'Lo siento, no tengo memoria de conversaciones anteriores. Cada pregunta es un nuevo comienzo.', 'extras': {'signature': 'CoQCAdHtim96BEr7xTeUIwcE5DxvT9OKGZnrnevOiMTXHQhd5SDE6ibXNZH7gWuOBWIoJaGUthfXMQ/9omAReXMgysW1O78NNyXravlTvB4F092OIKTcSKAdnZ8SkEqeydPJTUyBFB9+QPAkDSjF42LnZoq7Bnv8zj8CD/MtOh4c4jUtu5UF7EhQhMc0yMvv7IEv1hBWXT1+xbatn0yamo1pQGEtTN+Mj+fnoAXhoy2dJ7rNlZj9wnVkZt0VjTDUo/OtHqimJUhEGw4b0ziJ9ZtvsVceIydJye2LDO4sAlFIrfdas5BLlPXgjqZ5pqYYjPFacRVG5TY1Pq5K/niTswkb3rDm0Oc='}}], additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--110777c0-edd3-40cf-8e4a-ede815aa25f9-0', usage_metad

In [ ]:
checkpointer = MemorySaver()
agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)

def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable": {"thread_id": thread_id}}
    response = agent.invoke({"messages": [HumanMessage(query)]}, config=config)
    return response

print("\n Respuesta inicial")
resp1 = execute(agent, "Explícame la serie de Fourier", thread_id="a1b2c3")
print(resp1["messages"][-1].content)



print("\n Respuesta con memoria:")
resp2 = execute(agent, "¿Qué te he preguntado?", thread_id="a1b2c3")
print(resp2["messages"][-1].content)

/tmp/ipython-input-2711854824.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)



 Respuesta inicial
La serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). En otras palabras, descompone una función periódica en sus componentes de frecuencia más simples. Esto es útil porque permite analizar y manipular funciones complejas de una manera más sencilla.

 Respuesta con memoria:
Me preguntaste sobre la serie de Fourier.


In [ ]:
resp1 = execute(agent, "Recuerda que mi color favorito es azul.", thread_id="demo123")
print(resp1["messages"][-1].content)

resp2 = execute(agent, "¿Cuál es mi color favorito?", thread_id="demo123")
print(resp2["messages"][-1].content)

# Forzar olvido: nuevo thread_id
resp3 = execute(agent, "¿Cuál es mi color favorito?", thread_id="otro123")
print(resp3["messages"][-1].content)

[{'type': 'text', 'text': 'De acuerdo, recordaré que tu color favorito es azul.', 'extras': {'signature': 'CvQBAdHtim8h20HXvRtk60QYvzZnQ9jJ2ffAbJEdL1UQQU6eGDAJfuuXMupBIkgQ7nxFcZ4Y86Bb01Sw8HnZXE7pe7gZjJHQ2DUOarCU4RiBBLp146nUYfHKKQ3ddURDpjwsDfuUX4KaebV7iCZXubqMMfS1Y6f3nxCrag2j+rNdzoVea84cLc1LgXloIEoV3JICsXmJpbzMRmYz807L24MBFSzw2515ZB3f2/Btx18DqpuJUkS/Ol/haCypyq/fgdBu8SVEm7tPij1xZbyQ/rP9wcZWfqBXT+yJvu2+FBHz0AqyL6uXnSvbSwG91V3P20nddfOs+w=='}}]
Tu color favorito es azul.
[{'type': 'text', 'text': 'Lo siento, no tengo acceso a tu información personal, por lo que no puedo saber cuál es tu color favorito.', 'extras': {'signature': 'CqACAdHtim89wTWU+Kc5Emt0/EWq1kUdzUvJ2XmQmcpjAtPF+kj6H5GfgdVLTK9Zi0i/udKHTbleq0hwyb7/TlkgE5RVaPXlAyXZs4inm4uu6057jKLKsuqw+pwmUrPvt2A8fiGRLvy7cTKVNg2IUyTzBV8bw4YvYJupo/pt7W/tBALUJA3njknE3/PdIcCpZfFxS78n1/833aqRlw/5rqf+tengX7oU4ZlYe4uOMG+1I3o0zcoeK92cnWinEUYhfcgpqjxFj4s/yK3FQ0yEEfClFZYkDyJ8X0dMFLGdMum4mR4/Z2te+xsveyZ4bjhAeswgcfWtFI01FYE4Y8KDXhqM4Q/gGkb8GROcOTTy54gYrhvH

In [ ]:
from langsmith.run_trees import RunTree
from langsmith import Client, traceable

tracer = Client() # LangChainTracer has been deprecated, use Client instead.

# Decorador traceable para marcar la función de ejecución
@traceable(run_type="chain") # Specify run_type
def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable":{"thread_id": thread_id}} # Removed callbacks argument
    response = agent.invoke({"messages":[HumanMessage(query)]}, config=config)
    return response

# Ejecuta y luego entra a LangSmith para ver trace.
resp = execute(agent, "Explícame las series de Fourier")


print(resp["messages"][-1].content)

[{'type': 'text', 'text': 'Las series de Fourier son una herramienta matemática que nos permite descomponer cualquier función periódica (es decir, una función que se repite a intervalos regulares) en una suma de funciones trigonométricas más simples: senos y cosenos.\n\nImagina que tienes una onda compleja. La serie de Fourier te permite ver esa onda compleja como la suma de muchas ondas de seno y coseno más simples, cada una con su propia frecuencia y amplitud. Es como desarmar un sonido complejo en sus notas musicales individuales.\n\nEsto es muy útil en campos como la ingeniería, la física y el procesamiento de señales, porque al entender las frecuencias individuales que componen una señal, es más fácil analizarla, modificarla o incluso predecirla.', 'extras': {'signature': 'Cs4GAdHtim9ZZQzrFXpQnTbGuuo6P3J3Jtu3+gZdK5c4PpmgQUcPHKJ8dLyngdIcQ4YfbDaS+i12OxG7nsfu4iFwch47yBTvbQRjzffCPHbbV0VuRUGabpf4dONex15iPqRoWMCQfvOj8qm1qdi4EeBw0aPs0IAfj+oYWhF3L8P64ZORdFczXP5LF9+ictCEkRxykML2cfiyy1u4ij9